In [30]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import numpy as np
data = pd.read_csv(r"C:\Users\40103869\Downloads\latest_ADS.csv")
data = data.drop(['Unnamed: 0', 'finalspend_usd', "effectiveness_usd", "starttime"], axis = 1)

# Load the model
from catboost import CatBoostRegressor
loaded_model = CatBoostRegressor()


# MENTION THE ZONE
zone = "zone_NAZ"

loaded_model.load_model(rf"C:\Users\40103869\Downloads\model_Arun_zone_NAZ.cbm")
data = data[data[zone]==1]


In [31]:
data.shape

(505, 348)

In [32]:
data = data[loaded_model.feature_names_ + ["data","event_id", "efficiency"]]

In [33]:
train = data[data["data"]== 'train']
forecast = data[data['data'] == 'submission_df']

In [34]:
train.shape

(383, 26)

In [35]:
forecast.shape

(122, 26)

In [36]:
binary_variables = [
    'canparticipantsplacebidsduringpreviewperiod_Do not allow prebids',
    'improvebidamountby_Percentage',
    'showlineitemlevelrankinlot_Yes, to Buyers and Participants',
    'enabletrafficlightbidding_Yes',
    'setareviewperiodafterlotoritemcloses_Yes',
    'setareviewperiodafterlotcloses_Yes',
    'canparticipantsseeranks?_Their own rank',
    'canparticipantsseeranks?_No',
]

continuous_variables = [
    'reviewtimeperiod',
    'biddingperiod', 
    'overtimeperiod(minutes)',
    'bidrankthattriggersovertime',
]

other_variables = [
    'baselinespend_usd',
    # 'zone_APAC',
    'Count_supplier',
    'event_template_ABI Reverse Auction - NI',
    'categ_l2_IND CAPEX',
    'item_name',
    'Mean_Lot_Item_Bid_Version',
    'ratio',
    'itemno',
    'bestbid_usd',
    'participant',
    'Count_Item_Name'
]
X_forecast = forecast.drop(['data', 'event_id'], axis = 1)
X_forecast

,reviewtimeperiod,biddingperiod,overtimeperiod(minutes),bidrankthattriggersovertime,canparticipantsplacebidsduringpreviewperiod_Do not allow prebids,improvebidamountby_Percentage,"showlineitemlevelrankinlot_Yes, to Buyers and Participants",enabletrafficlightbidding_Yes,setareviewperiodafterlotoritemcloses_Yes,setareviewperiodafterlotcloses_Yes,...,Count_Item_Name,ratio,Count_supplier,participant,bestbid_usd,Mean_Lot_Item_Bid_Version,itemno,baselinespend_usd,item_name,efficiency
15025,0.0,10.0,1.0,10.0,1,0,0,1,0,0,...,349812,1.0,349812,1.0,97349.97,77.891784,1.0,97349.97,1,0.0
15081,2.0,20.0,0.0,0.0,1,0,0,1,1,0,...,440097,1.0,440097,1.0,230674.68,108.800247,1.0,230674.68,1,0.0
15094,2.0,30.0,2.0,4.0,1,0,0,1,0,1,...,440097,3.0,440097,3.0,275000.00,108.800247,1.0,275000.00,1,0.0
15131,2.0,10.0,2.0,2.0,1,0,0,1,1,0,...,349812,2.0,349812,2.0,221197.00,77.891784,1.0,221197.00,1,0.0
15170,2.0,30.0,5.0,2.0,0,0,0,1,0,1,...,349812,3.0,349812,3.0,72993.00,77.891784,1.0,72993.00,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19459,2.0,30.0,5.0,2.0,0,0,0,0,0,1,...,349812,6.0,349812,6.0,2952000.00,77.891784,1.0,2952000.00,1,0.0
19507,2.0,15.0,2.0,3.0,1,0,0,1,0,1,...,349812,4.0,349812,4.0,145537.00,77.891784,1.0,145537.00,1,0.0
19515,2.0,15.0,0.0,0.0,1,0,0,1,1,0,...,170891,1.0,170891,5.0,2536153.62,219.372272,5.0,2847500.00,5,0.0
19581,2.0,0.0,2.0,1.0,1,0,0,1,1,0,...,440097,1.0,440097,1.0,1567500.00,108.800247,1.0,1567500.00,1,0.0


In [37]:
from scipy.optimize import minimize
import numpy as np

def objective_function_scipy(vars, other_values, loaded_model, continuous_count, binary_count):
    """
    Objective function for optimization.
    """
    continuous_vars = vars[:continuous_count]
    binary_vars = vars[continuous_count:]
    # binary_vars = np.round(vars[continuous_count:])  # Ensure binary variables are rounded to 0 or 1
    candidate = list(continuous_vars) + list(binary_vars) + other_values
    # print(candidate)
    # print(loaded_model.predict(candidate))
    return -loaded_model.predict(candidate)  # Negate because we maximize in Pyomo but SciPy minimizes

def scipy_optimization(x_row):
    """
    Optimization using SciPy.
    """
    # Extract problem data
    other_values = list(x_row[other_variables])  # Fixed values for other variables
    continuous_count = len(continuous_variables)
    binary_count = len(binary_variables)
    
    # Define bounds for variables
    bounds = [(0, 10), (0,100), (0,10), (0,5)] + [(0, 1)] * binary_count  # Continuous [0, 120], Binary [0, 1]
    
    # Define initial guesses
    x0 = list(x_row[continuous_variables + binary_variables])
    # x0 = [0] * len(continuous_variables + binary_variables)
    # x0 = list(x_row[continuous_variables]) + [0.5] * binary_count

    # Define constraints
    constraints = [
        {'type': 'ineq', 'fun': lambda vars: 10 - vars[continuous_count + 0]},  # y[0] <= 10
        {'type': 'ineq', 'fun': lambda vars: 100 - vars[continuous_count + 1]},  # y[1] <= 100
        {'type': 'ineq', 'fun': lambda vars: 10 - vars[continuous_count + 2]},  # y[2] <= 10
        {'type': 'ineq', 'fun': lambda vars: 5 - vars[continuous_count + 3]},  # y[3] <= 5
        {'type': 'ineq', 'fun': lambda vars: 1 - vars[continuous_count + 6] - vars[continuous_count + 7]},  # y[3] <= 5
    ]
    constraints = constraints + [{'type' : 'ineq', 'fun' : lambda vars: 1 - vars[continuous_count + i]} for i in range(binary_count)]

    # Solve using SciPy
    result = minimize(
        objective_function_scipy,
        x0=x0,
        args=(other_values, loaded_model, continuous_count, binary_count),
        bounds=bounds,
        constraints=constraints,
        # method='SLSQP',  # Sequential Least Squares Programming
        method = "COBYLA",
        options={'disp': True}
    )

    # Extract results
    optimized_vars = result.x
    continuous_vars = np.round(optimized_vars[:continuous_count])
    binary_vars = np.round(optimized_vars[continuous_count:])  # Round binary variables

    # Display results
    print("Optimization Result:")
    print("Continuous Variables:", continuous_vars)
    print("Binary Variables:", binary_vars)
    print("initial vars: ", list(x0))

    best_candidate = list(continuous_vars) + list(binary_vars) + other_values
    result_objective = loaded_model.predict(best_candidate) 
    print("Objective Value:", result_objective)

    return result_objective, list(continuous_vars) + list(binary_vars)

In [38]:
optimized_effienciencies = []
optimized_x = []
print(X_forecast.shape[0])
for i in range(X_forecast.shape[0]):
# for i in range(10):
    x_row = X_forecast.iloc[i]
    print(X_forecast.shape[0] - i)
    best_eff, best_x = scipy_optimization(x_row)
    optimized_effienciencies.append(best_eff)
    optimized_x.append(best_x)

122
122
Optimization Result:
Continuous Variables: [ 0. 10.  3.  4.]
Binary Variables: [1. 1. 1. 1. 0. 1. 0. 1.]
initial vars:  [0.0, 10.0, 1.0, 10.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]
Objective Value: 0.12703564654755942
121
Optimization Result:
Continuous Variables: [ 2. 20.  0.  1.]
Binary Variables: [1. 1. 1. 1. 1. 0. 0. 1.]
initial vars:  [2.0, 20.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0]
Objective Value: 0.11948519376551951
120
Optimization Result:
Continuous Variables: [ 2. 30.  3.  3.]
Binary Variables: [ 1.  1.  0.  1.  0.  1. -0.  1.]
initial vars:  [2.0, 30.0, 2.0, 4.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0]
Objective Value: 0.1334095667923583
119
Optimization Result:
Continuous Variables: [ 2. 10.  3.  2.]
Binary Variables: [1. 0. 1. 1. 1. 1. 0. 1.]
initial vars:  [2.0, 10.0, 2.0, 2.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0]
Objective Value: 0.12084911083343874
118
Optimization Result:
Continuous Variables: [ 2. 30.  5.  3.]
Binary Variables: [1. 1. 0. 1. 

In [40]:
y_pred = loaded_model.predict(X_forecast)
y_pred

array([5.40548763e-02, 7.41360755e-02, 1.08822097e-01, 1.11401771e-02,
       8.75892628e-02, 4.72841041e-02, 4.60785093e-02, 4.79694886e-02,
       2.18244732e-02, 3.32432873e-01, 8.30479927e-02, 1.44236684e-01,
       3.80882723e-03, 3.99539419e-02, 8.65497996e-02, 1.16854247e-01,
       3.50302674e-02, 3.89730362e-02, 2.39775154e-02, 6.70364957e-02,
       2.76020411e-02, 7.67159065e-02, 3.04306344e-02, 1.28439555e-01,
       9.13193094e-02, 1.06043660e-01, 6.69020142e-02, 9.55012485e-02,
       5.51367938e-02, 8.50172102e-02, 6.16681719e-02, 7.05640147e-02,
       5.41264058e-02, 1.86328194e-01, 1.15769403e-01, 6.40421262e-02,
       6.90686009e-02, 1.30402554e-01, 5.64277995e-02, 6.53815622e-02,
       6.03305715e-02, 9.18724664e-02, 9.21999452e-02, 5.00646237e-02,
       3.90178017e-02, 2.85766529e-01, 1.02230464e-01, 1.88173448e-01,
       2.23979532e-02, 1.72627900e-01, 2.63314584e-03, 1.26457204e-04,
       5.65090302e-02, 8.47117092e-02, 8.22237902e-02, 1.56878956e-02,
      

In [41]:
final = pd.DataFrame(optimized_x, columns = loaded_model.feature_names_[:len(continuous_variables) + len(binary_variables)])
final_1 = pd.concat([final, forecast[other_variables].reset_index(drop = True)], axis = 1)
final_1['event_id'] = forecast['event_id']
final_1['Optimized_efficiency'] = optimized_effienciencies
final_1['Initial_efficiency'] = y_pred
final_1['Uplift'] = final_1['Optimized_efficiency'] - final_1['Initial_efficiency']
final_1['Uplift_dollars'] = final_1['Uplift'] * final_1['bestbid_usd'] * final_1['item_name']

In [42]:
final.shape

(122, 12)

In [43]:
print(f"Uplift in effiency : {sum(final_1['Uplift_dollars']/sum(final_1['bestbid_usd'] * final_1['item_name']))*100} %")

Uplift in effiency : 4.217683941064672 %


In [44]:
final_1.shape

(122, 28)

In [49]:
final_1=pd.read_csv(r'C:\Users\40103869\OneDrive - Anheuser-Busch InBev\Fortress\hackathon2\Notebooks\Optimized_result_Arun_Model_scipy_zone_APAC.csv')

In [50]:
final_1['final_spend'] =(1- final_1['Initial_efficiency']) * final_1['bestbid_usd']
final_1['final_spend_optimized'] =(1- final_1['Optimized_efficiency']) * final_1['bestbid_usd']
final_1['Uplift_dollars'] = final_1['Uplift'] * final_1['bestbid_usd']
print(f"Uplift in effiency : {sum(final_1['Uplift_dollars']/sum(final_1['bestbid_usd']))*100} %")

Uplift in effiency : 5.588647418661121 %


In [51]:
final_1

,reviewtimeperiod,biddingperiod,overtimeperiod(minutes),bidrankthattriggersovertime,canparticipantsplacebidsduringpreviewperiod_Do not allow prebids,improvebidamountby_Percentage,"showlineitemlevelrankinlot_Yes, to Buyers and Participants",enabletrafficlightbidding_Yes,setareviewperiodafterlotoritemcloses_Yes,setareviewperiodafterlotcloses_Yes,...,bestbid_usd,participant,Count_Item_Name,event_id,Optimized_efficiency,Initial_efficiency,Uplift,Uplift_dollars,final_spend,final_spend_optimized
0,2.0,20.0,6.0,4.0,1.0,1.0,1.0,1.0,1.0,0.0,...,40823.71,3.0,440097,NaN,0.160151,0.159647,0.000504,20.579390,3.430632e+04,3.428574e+04
1,3.0,30.0,6.0,5.0,0.0,1.0,1.0,0.0,1.0,0.0,...,25644.36,4.0,440097,NaN,0.182693,0.102915,0.079777,2045.838232,2.300516e+04,2.095933e+04
2,2.0,30.0,5.0,5.0,0.0,1.0,1.0,1.0,1.0,1.0,...,16253.81,4.0,440097,NaN,0.185413,0.123158,0.062254,1011.871529,1.425202e+04,1.324015e+04
3,2.0,15.0,6.0,3.0,0.0,1.0,1.0,1.0,1.0,1.0,...,12962.38,3.0,440097,NaN,0.162623,0.032398,0.130224,1688.015980,1.254242e+04,1.085440e+04
4,2.0,0.0,4.0,5.0,1.0,1.0,1.0,0.0,0.0,1.0,...,1414700.00,27.0,311963,NaN,0.152474,0.167413,-0.014939,-21134.268792,1.177861e+06,1.198995e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2517,5.0,30.0,5.0,4.0,-0.0,1.0,1.0,-0.0,1.0,1.0,...,114415.24,4.0,349812,NaN,0.180111,0.149311,0.030800,3523.962884,9.733174e+04,9.380778e+04
2518,1.0,0.0,4.0,3.0,0.0,1.0,0.0,1.0,1.0,0.0,...,245308.97,12.0,440097,NaN,0.188241,0.140517,0.047724,11707.123820,2.108388e+05,1.991317e+05
2519,3.0,30.0,7.0,5.0,0.0,1.0,1.0,0.0,1.0,1.0,...,104523.75,3.0,349812,NaN,0.185035,0.243188,-0.058153,-6078.376043,7.910486e+04,8.518323e+04
2520,3.0,30.0,5.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,3442025.08,12.0,349812,NaN,0.172236,0.126214,0.046022,158409.659095,3.007595e+06,2.849185e+06


In [53]:
final_1.to_csv(f"Optimized_result_Arun_Model_scipy_APAC1.csv", index = False)